## Segmentation and Cluster of Neighborhoods in Toronto

Use the Notebook to build the code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe like the one shown at <a href="https://d3c33hcgiwev3.cloudfront.net/imageAssetProxy.v1/7JXaz3NNEeiMwApe4i-fLg_40e690ae0e927abda2d4bde7d94ed133_Screen-Shot-2018-06-18-at-7.17.57-PM.png?expiry=1588809600000&hmac=K2IKamXvUjuNmLLVhDi_Wu2GdwvfoENh4B5fk0IWC6U">the coursera site.</a>
https://d3c33hcgiwev3.cloudfront.net/imageAssetProxy.v1/7JXaz3NNEeiMwApe4i-fLg_40e690ae0e927abda2d4bde7d94ed133_Screen-Shot-2018-06-18-at-7.17.57-PM.png?expiry=1588809600000&hmac=K2IKamXvUjuNmLLVhDi_Wu2GdwvfoENh4B5fk0IWC6U

To create the above dataframe:


    


In [43]:
conda install -n myenv -c conda-forge geopy --yes


Solving environment: ...working... done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [38]:
conda install -n myenv -c conda-forge beautifulsoup4 --yes


Solving environment: ...working... done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [39]:
conda install -n myenv -c conda-forge lxml --yes

Solving environment: ...working... done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [ ]:
CLIENT_ID = 'C3NLZY0MJGCJENLKP4I5R0NONFIS3A4DPRNLGV33MWAU5Y3O' # your Foursquare ID
CLIENT_SECRET = '2OOM4WNGBAPZ5M0NC0E4JF31EZXS2KD2XQETJVHAIGTOWMZM' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails are hidden as CLIENT_ID, CLIENT_SECRET.')
#print('CLIENT_ID: ' + CLIENT_ID)
#print('CLIENT_SECRET:' + CLIENT_SECRET)

The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
--Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated 
with a comma as shown in row 11 in the above table.

--If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.


In [66]:
from bs4 import BeautifulSoup
import requests

source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

#with open("List of postal codes of Canada  M - Wikipedia.html", encoding='utf8') as html:
#    soup = BeautifulSoup(html)
soup = BeautifulSoup(source)
#print (soup)
t = soup.table
table_data = []
t_headers = ['PostalCode', 'Borough', 'Neighborhood']

for tr in t.tbody.find_all("tr"): # find all tr's from table's tbody
    t_value = []
    # Each table row is stored in the form of
    #t_row = {'Postal code', 'Borough', 'Neighborhood(s)'}
    # find all td's(3) in tr and zip it with t_header
    #for td, th in zip(tr.find_all("td"), t_headers): 
    #    t_row[th] = td.text.replace('\n', '').strip()
    for td in tr.find_all('td'):
        t_value.append(td.text.replace('\n', '').strip())
    if t_value:
        table_data.append(t_value)

mydata = table_data
#print (mydata[0:10])
cleaned = {v[0]: v for v in mydata if v and v[1]!='Not assigned' }
#print (cleaned)
for k in cleaned:
    cleaned[k][2] = cleaned[k][2].replace(' / ', ', ')
    if cleaned[k][2] == 'Not assigned': cleaned[k][2] = cleaned[k][1]

for a in list(cleaned.keys())[0:10]:
     print (cleaned[a])



['M3A', 'North York', 'Parkwoods']
['M4A', 'North York', 'Victoria Village']
['M5A', 'Downtown Toronto', 'Regent Park, Harbourfront']
['M6A', 'North York', 'Lawrence Manor, Lawrence Heights']
['M7A', 'Downtown Toronto', "Queen's Park, Ontario Provincial Government"]
['M9A', 'Etobicoke', 'Islington Avenue']
['M1B', 'Scarborough', 'Malvern, Rouge']
['M3B', 'North York', 'Don Mills']
['M4B', 'East York', 'Parkview Hill, Woodbine Gardens']
['M5B', 'Downtown Toronto', 'Garden District, Ryerson']


In [75]:
import pandas as pd

t_headers = ['PostalCode', 'Borough', 'Neighborhood']
data = pd.DataFrame.from_dict(cleaned, orient='index', columns=t_headers)

data.head()


,PostalCode,Borough,Neighborhood
M3A,M3A,North York,Parkwoods
M4A,M4A,North York,Victoria Village
M5A,M5A,Downtown Toronto,"Regent Park, Harbourfront"
M6A,M6A,North York,"Lawrence Manor, Lawrence Heights"
M7A,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [72]:
data.shape

(103, 3)